<a href="https://colab.research.google.com/github/Firojpaudel/GenAI-Chronicles/blob/main/Transformers_groundup_learnings/model_py_explained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is just the breakdown of model.py file inside the `Working_full_fledged` folder.

## Learning the Transformers architecture in detail

---

<div align="center">
    <img src= "https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png" alt="The transformers architecture" width="250px">
    <p><i>The Transformers Architecture from the OG paper</i></p>
</div>

I have a foundational understanding of the Transformer architecture from the _'Attention is All You Need'_ paper, but now I'm diving deeper into the concepts. Currently, I'm learning from **Umair Jamil** Sir's 'Coding a Transformers from Scratch' course on PyTorch with **Jay Alamar**'s blog combined.

Fingers crossed everything goes smoothly! 🤞

---

Starting with the `input_embedding`Layer:

<div align="center">
    <img src= "https://images.ctfassets.net/k07f0awoib97/2n4uIQh2bAX7fRmx4AGzyY/a1bc6fa1e2d14ff247716b5f589a2099/Screen_Recording_2023-06-03_at_4.52.54_PM.gif" alt="Input Emedding" width="550px">
    <p><i>The Embedding Mechanism illustrated</i></p>
</div>

In [2]:
##@ All required imports are here in this cell
import warnings
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq

In [ ]:
warnings.filterwarnings('ignore')

---

### 1. Building up the Architecture

In [3]:
class InputEmbeddings(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        "The embedding is done in pytorch using just a simple nn.Embedding function"
        self.embed= nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        return self.embed(x) * torch.sqrt(self.d_model, dtype=torch.float32)

Okay, so the `Embedding` was already done earlier but we scale these embeddings while passing forward, for majorly two reasons:

1. The `d_model` scaling is done inorder to ensure the magnitude of input embedding and positional encoding are appropriately balanced.
2. Also, it maintains the informational integrity.

> _src: "StackOverflow"_

_Well Mathematically_,

Let's assume:

- Input embeddings as $\text{E}$
- Positional encodings as $\text{P}$

Then, the combined input to the model would be: $$\text{X} = \sqrt{d_{model}} \times \text{E} + \text{P}$$


Ofc, it will go through the `softmax` function before that but yeah..

This scaling ensures that the variance of the embeddings is in line with the variance of the positional encodings, leading to more stable training and better convergence.

After `input_embedding`, time for `positional_encoding`

In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, seq_length: int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.seq_length = seq_length
        self.dropout = nn.Dropout(dropout)

        ## Create a positional encoding matrix of size seq_length x d_model
        pe = torch.zeros(seq_length, d_model)
        ## Create a vector of shape (seq_length, 1)
        pos = torch.arange(0, seq_length).unsqueeze(1)  ## The numerator part
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model)) ## The denominator part

        ## Apply sine to even positions
        pe[:, 0::2] = torch.sin(pos * div)

        ## Apply cosine to odd positions
        pe[:, 1::2] = torch.cos(pos * div)

        ## Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0)

        ## Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x= x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x)

_Explaining the code above:_

> "  Since our model contains no recurrence and no convolution, in order for the model to make use of the
 order of the sequence, **we must inject some information about the relative or absolute position of the
 tokens in the sequence**. To this end, we add "positional encodings" to the input embeddings at the
 bottoms of the encoder and decoder stacks. **The positional encodings have the same dimension dmodel
 as the embeddings**, so that the two can be summed. There are many choices of positional encodings,
 learned and fixed "

 *-src: paper*

Here the authors of this paper have clearly mentioned, first we should inject a sequence and ofc `seq_length` needs to be defined.
Next, the dimension we used earlier in the `input_embeddings` is to be used. And, finally there's a `dropout` which is added in order
to prevent the model from overfitting.


Then again from the same subsection, the `positional_encoding` formulas are provided. We use different formulas for odd and even dimensions.

The parameters are same, we just pass through different funcitons.

- $sin$ for even dimensions and
- $cos$ for odd dimensions.

i.e., $$ \text{PE}_{(pos, 2i)} = sin(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$$
$$ and, \text{PE}_{(pos, 2i+1)} = cos(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$$

So, the numerators and denominators are defined before passing through these functions.

_Numerator part first:_
```python
pos = torch.arange(0, seq_len).unsqueeze(1)
```

Here, the position `pos` is unsquezed to 1, for column vector formatting of shape (seq_length, 1)

_Denominator part:_

```python
div = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)))/ d_model)
```

Well, it was meant to be $ 10000 ^{\frac{2i}{d_{model}}}$ but well in coding implementation, we could use log instead.

$$ 10000 ^{\frac{2i}{d_{model}}} \equiv  exp. (ln(10000) \cdot \frac{2i}{d_{model}})$$

Also, we are using negative log to ensure the scaling decreases progressively as $i$ increases


Next, we add batch dimension to make the positional encoding `pe` compatible with **batch_processing**. The resulting shape is `(1, seq_len, d_model)`.

Then, there comes registering buffers... We register the `pe` as a buffer in the `PositionalEncoding` module so that the `pe` is not treated as paprameter but
is still part of the state of the module. It's useful for saving and loading models.

---

Next, entering inside the box; Coding the easiest one of them first.. **Add & Norm** layer aka. _Layer Normalization_

**Layer Normalization**: Technique to stabilize and speed up the training of NNs.

Let's assume _a batch_ of 'n' items. Each with mean $\mu$ and variance $\sigma ^2 $.

Now for normalizing the items in this batch, we calculate the new value as: $$ \hat{x_{i}} = \frac{x_{i} - \mu}{\sqrt{\sigma ^2 + \epsilon}}$$

where, $\epsilon$ is a small constant to prevent the division by zero.

Then, there comes scaling and shifting. aka. the multiplicative and additive steps.

After normalization, the model applies **learnt parameters** $\gamma$(scaling) and $\beta$(shifting) as: $$y_{i} = \gamma \cdot \hat{x_i} + \beta$$

And, this looks so much similar to the the equation of **fully connected layer**. ie., $$y= W \cdot x + b$$

_More on these learnt parameters:_

- $\gamma$ — also known as _multiplicative_ or _scaling_ scales (stretch/shrink) the normalized value $\hat{x_i}$.
    - if $\gamma < 1$, it compresses,
    - if $\gamma > 1$, it amplifies.
- $\beta$ — also known as _additive_ or _shifting_ shifts the scaled normalized values up and down. **It allows the network to "center" the activations where needed.**

> _The $\gamma$ and $\beta$ parameters allow flexibility, ensuring the network can adapt the normalized values when necessary._

In [5]:
class LayerNormalization(nn.Module):
    def __init__(self, epsilon: float = 10**-6):
        super().__init__()
        self.epsilon = epsilon
        self.gamma = nn.Parameter(torch.ones(1))  # The scaler
        self.beta = nn.Parameter(torch.zeros(1))  # The shifter aka bias

    def forward(self, x):
        mean = x.mean(dim= -1, keepdim=True)
        std = x.std(dim= -1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.epsilon) + self.beta

_Code Explanation incoming:_

Okay, so first we had variance in theory part. How come in code, we have standard deviation??

Well, standard deviation is just sqrt of variance and the epsilon value, we can just put the square rooted value any way.
This way, the coding becomes easier with same functionality.

ie., $\text{standard_deviation} = \sqrt{variance}$

Then, we initialize the value of learnable parameter $\gamma$ to 1 and $\beta$ to zero

---

Next stop, let's expolore the **Feed Forward Layer**:

From the paper,
<div align="center">
    <img src= "./screen_shots_from_the_OG_paper/Feed_forward.jpg" width="600px">
    <p><i>SS from the paper</i></p>
</div>

First we will calculate the linear transformation—1 by calculating $xW_{1} + b_{1}$ then pass this transformation into $RELU$ activation and then the output acts as input in the next linear transformation—2 as:
$$ \text{FFN}(x) = max(0, xW_{1} + b_{1})W_{2} + b_{2}$$

In [6]:
class FeedForward(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # The first linear layer with W1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # The second linear layer with W2 and b2

    def forward(self, x):
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

| Description       | Details                                                        |
|-------------------|----------------------------------------------------------------|
| In Linear Layer 1 | **Input_dimension**: d_model ; **Output_dimension**: d_ff (inner_layer)|
| In Linear Layer 2 | **Input_dimension**: d_ff ; **Output_dimension**: back to d_model      |


---
And finally, turn for the most important block; **Multi-Head Attention Block**

Before diving into the Multi-Head Attention Block, we need to know about the Scaled Dot-product attention mechanism.

**Scaled Dot-Product Mechanism**:

<div align="center">
    <img src="https://github.com/Firojpaudel/GenAI-Chronicles/blob/main/Transformers_groundup_learnings/screen_shots_from_the_OG_paper/Scaled-dot_product_attn.jpg?raw=1" width="250px">
    <p><i>Figure representing Scaled Dot-product Attention</i></p>
</div>

_Explaining the process:_

1. **Sentence Breakdown**:
    - Start with any sentence. This sentence is broken down into "Query", "Keys", and "Values".

2. **Query-Key Multiplication**:
    - Multiply the Query and Keys. This multiplication helps in determining the relevance of each key with respect to the query.

3. **Scaling**:
    - Scale the multiplied values. This step ensures that the values are within a manageable range, preventing extremely large values that could destabilize the training process.

4. **Masking (Optional)**:
    - Pass the scaled values through a masking layer. Masking is used to prevent attending to certain positions, typically in the context of sequence-to-sequence models.

5. **Activation Function**:
    - Apply the activation function (softmax, as mentioned in the paper). This step converts the scaled values into probabilities, highlighting the most relevant keys.

  - Given by formula: $$\text{Attention}(Q,K,V) = \text{softmax}(\frac{QK^T}{\sqrt{d_{k}}})V $$

        where,

        - $d_{k}$ — dimension of keys == dimension of queries (so they can be multiplied duhh.. ) &&

        - $d_{v}$ — dimension of values.

6. **Final Multiplication**:
    - Multiply the output of the activation function with the Values. This step produces the `Attention(Q, K, V)` value, which is a weighted sum of the values based on the relevance scores.



**Multi-Head Attention**

<div align="center">
    <img src="https://github.com/Firojpaudel/GenAI-Chronicles/blob/main/Transformers_groundup_learnings/screen_shots_from_the_OG_paper/Multihead.jpg?raw=1" width="250px">
    <p><i>Figure Representing the Multi-Head Attention Process</i></p>
</div>

- In Scaled Dot-Product Attention, we focus on a single set of queries, keys, and values. However, with Multi-Head Attention, we apply this mechanism across multiple sets of queries, keys, and values, enabling the model to capture different features from the input sequence.

- Each set of queries, keys, and values is linearly projected before being fed into the Scaled Dot-Product Attention layer. This allows each head to focus on different parts of the input, enhancing the model's ability to understand complex patterns.

In [7]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, h: int, dropout: float):
        super().__init__()
        self.d_model= d_model
        self.h= h
        assert d_model % h ==0, "d_model is not divisible by h"

        self.d_k = d_model // h
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)

        self.w_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]

        attention_scores = query @ key.transpose(-2, -1) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))
        if mask is not None:
            attention_scores.masked_fill(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim = -1)

        if dropout is not None:
            attention_scores = dropout(attention_scores)

        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)

        query = query.view(query.size[0], query.shape[1], self.h, self.d_k).transpose(1,2)
        key = key.view(key.size[0], key.shape[1], self.h, self.d_k).transpose(1,2)
        value = value.view(value.size[0], value.shape[1], self.h, self.d_k).transpose(1,2)

        x, self.attention_scores = MultiHeadAttention.attention(query, key, value, mask, self.dropout)

        ## (Batch_size, h, seq_length, d_k) -> (Batch_size, seq_length, h, d_k) -> (Batch_size, seq_length, d_model)
        x= x.transpose(1, 2).contiguous().view(x.size(0), -1, self.h * self.d_k)  #@ self.h * self.d_k = d_model

        #@ Now finally we multiply this with the output weights:
        return self.w_o(x)

_Code Explanation of above Cell:_

- **Starting from `h`**: `h` is the number of attention heads. The assertion ensures `d_model` is divisible by `h` so that each head gets an equal portion of model dimension.
- `d_k`: The dimension of each attention head.
- `w_q, w_k, w_v`: Are the linear layers to project input queries(Q), keys(K) and values(V) from `d_model` to `d_model`.
- `w_o`: is the linear layer to combine the outputs of attention heads

- Next, we have defined the attention function statically,

    Reason behind defining with `@staticmethod`:
    - **No Dependency on Class Instance**: The attention function doesn't need any instance-specific data or attributes to perform its operations. It relies solely on the inputs provided (`query`, `key`, `value`, `mask`, and `dropout`). Therefore, it can be defined as a static method because it doesn't require access to the instance (self) of the class.
    - **Reusability**: Static methods can be called without needing to create an instance of the class. This makes the attention function more flexible and reusable in different contexts, even outside the `MultiHeadAttention` class if needed.

- We’ve reintroduced the dimension d_k inside the static function because the static function acts independently. This time, we simply extract the last value from the query shape as d_k, which represents the dimension of the embedded vectors. This works because it’s just a matrix multiplication after all. 🤔 _(The figure at the end of this cell will definitely help us understand that)_

- Then, in `atttention_scores`, we have just applied the formula $ \frac{Q K^T}{\sqrt{d_{k}}}$. Also, transposing the second last and last positions.

> PS: If you are wondering what @ is, learn python haiyaa.. why are you even here? ✋😕🤚

- Likewise in the same `attention_score`, if there is mask provided, masking very large negative values to 0, just ignoring...

- Then, pass it through the softmax activation

- Also,if the dropout exists, apply it in `attention_scores`. It would prevent overfitting.

- Then matrix multiplication with values and attention score itself.

_In formard function:_

- We just passed query,key and value through the respective linear layers.

Now explaining this section:
```python
query = query.view(query.size[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
key = key.view(key.size[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
value = value.view(value.size[0], value.shape[1], self.h, self.d_k).transpose(1, 2)
```

Okay so typically, the shape/size of `q,k and v` are in the format: `(Batch_size, seq_len, d_model)`. Now, we are just changing the view to be in the format of : `(Batch_size, seq_len, h, d_k)`

Then after transposing 1 and 2 positions, we get the shape of: `(Batch_size, h, seq_len, d_k)`

But the major question is why change the view?
Answer to that is here:

1. **Multi-Head Attention Mechanism:** In the Transformer model, the attention mechanism is split into multiple heads. This allows the model to focus on different parts of the input sequence simultaneously. Each head has its own set of weights and performs its own attention calculation.

2. **Reshaping for Parallel Computation:** The original shape of the tensors `query, key, and value` is `(Batch_size, seq_len, d_model)`. `d_model` is the dimension of the model, which is split into `h` heads, each with a dimension of `d_k` (where $d_{k} = \frac{d_{model}}{h}$). By reshaping the tensors to `(Batch_size, seq_len, h, d_k)`, we prepare them for parallel computation across the multiple heads.

3. **Transposing for Efficient Computation:** After reshaping, we transpose the tensors to `(Batch_size, h, seq_len, d_k)`.
This transposition ensures that the heads are the second dimension, which allows for efficient computation of attention scores across the heads.

Then lets talk about  the contiguous memory allocation. The `.contiguous()` function ensures that the tensor's memory layout is suitable for the next operation `(view)`. This step is necessary because transpose can result in non-contiguous memory storage, which is incompatible with reshaping.

<div align="center">
    <img src="https://github.com/Firojpaudel/GenAI-Chronicles/blob/main/Transformers_groundup_learnings/screen_shots_from_the_OG_paper/Mechanism_explained.png?raw=1">
    <p><i>Multihead Attention visually explained</i></p>
    <p><i>*Courtesy: Umair Jamil sirs video*</i></p>
</div>

---

Well, now finally, we define the **residual_connections** or the **skip_connections**,

In [8]:
class ResidualConnection(nn.Module):
    def __init__(self, dropout: float):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization()

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

All the required functions for the Encoder/Decoder blocks are done. Now, we just need to define the Encoder block...

---

In [9]:
class EncoderBlock (nn.Module):
    def __init__(self, self_attn: MultiHeadAttention, feed_forward: FeedForward, dropout: float):
        super().__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.residual_connection = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

    def forward(self,x, mask):
        ''' why apply mask here?:
        The mask is applied to the self attention mechanism to prevent the model from attending to the future tokens.'''
        x= self.residual_connection[0](x, lambda x: self.self_attn(x, x, x, mask))
        x= self.residual_connection[1](x, self.feed_forward)
        return x

_Code Explanations:_

So I was gonna use
```python
    self.residual_connection_1 = ResidualConnection(dropout)
    self.residual_connection_2 = ResidualConnection(dropout)
```
but well found out that using ModuleList is more appropriate for this kind of things...

Now, I'd like to explain the forward function in this module:
```python
x= self.residual_connection[0](x, lambda x: self.self_attn(x, x, x, mask))
x= self.residual_connection[1](x, self.feed_forward)
```
So here, first lets bring in the figure to explain this more clearly.

<div align= "center">
    <img src= "./screen_shots_from_the_OG_paper/Encoder_Block_zoomed.jpg" width= "300|px">
    <p><i>Encoder Block Zoomed with labelled Skip/Residual Connections</i></p>
</div>

The first residual or skip connection is defined as `[0]`, and the lambda function is used to pass arguments to the self_attn function dynamically. Specifically, it allows the self-attention layer to take $Q,K,V$ `(query, key, value)` all as $x$, along with the mask.

The second residual connection, `[1]`, wraps the feed-forward block, applying it directly to the output from the first residual connection.

---

Now defining the Encoder as a whole:


In [10]:
class Encoder(nn.Module):
    def __init__(self, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, mask):
        for layer in self.layers:
            x= layer(x, mask)
        return self.norm(x)  #Since after every layer we have Add and Norm layer.. 🤷

And with this encoder layer is pretty much done...

---

Next, lets go for the **decoder block**

In [11]:
class DecoderBlock(nn.Module):
    def __init__(self, self_attn: MultiHeadAttention, cross_attn: MultiHeadAttention, \
                 feed_forward: FeedForward, dropout: float):
        super().__init__()
        self.self_attn = self_attn
        self.cross_attn = cross_attn
        self.feed_forward = feed_forward
        self.residual_connection = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

    def forward(self, x, enc_output, src_mask, trgt_mask):
        x = self.residual_connection[0](x, lambda x: self.self_attn(x, x, x, trgt_mask))
        x = self.residual_connection[1](x, lambda x: self.cross_attn(x, enc_output, enc_output, src_mask))
        x = self.residual_connection[2](x, self.feed_forward)
        return x

In [12]:
''' Now coding the Decoder class '''

class Decoder(nn.Module):
    def __init__(self, layers: nn.ModuleList):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, enc_output, src_mask, trgt_mask):
        for layer in self.layers:
            x = layer(x, enc_output, src_mask, trgt_mask)
        return self.norm(x)

---

Now that the encoder and decoder are done, in next step we will define the **Linear Projection** layer after decoder.

<div align="center">
    <img src="https://github.com/Firojpaudel/GenAI-Chronicles/blob/main/Transformers_groundup_learnings/screen_shots_from_the_OG_paper/ProjectionLayer.jpg?raw=1" width="250px">
    <p><i>The Linear Projection Layer</i></p>
</div>

In [13]:
class ProjectionLayer(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.projection = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        ''' Also  the softmax is applied here in forward method'''
        return torch.log_softmax(self.projection(x), dim=-1)

---

Finally, time to implement the **Transformers** now that the architecture is defined



In [14]:
class Transformers(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, \
                 trgt_embed: InputEmbeddings, src_pos: PositionalEncoding, trgt_pos:PositionalEncoding,\
                 projection: ProjectionLayer):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.trgt_embed = trgt_embed
        self.src_pos = src_pos
        self.trgt_pos = trgt_pos
        self.projection = projection

    def encode(self, src, src_mask):
        src= self.src_embed(src)
        src= self.src_pos(src)
        return self.encoder(src, src_mask)

    def decode(self, trgt, enc_output, src_mask, trgt_mask):
        trgt= self.trgt_embed(trgt)
        trgt= self.trgt_pos(trgt)
        return self.decoder(trgt, enc_output, src_mask, trgt_mask)

    def project(self, x):
        return self.projection(x)

In [15]:
def build_transformer(src_vocab_size: int, trgt_vocab_size: int, src_seq_len: int,\
                    trgt_seq_len: int, d_model: int = 512, N: int = 6, h: int = 8,\
                    dropout: float = 0.1,d_ff: int = 2048)  -> Transformers:
    ## Create the embedding layers:
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    trgt_embed = InputEmbeddings(d_model, trgt_vocab_size)

    ## Positional  Encoding layers:
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    trgt_pos = PositionalEncoding(d_model, trgt_seq_len, dropout)

    ## Create the encoder and decoder blocks:
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention = MultiHeadAttention(d_model, h, dropout)
        feed_forward= FeedForward(d_model, d_ff, dropout)
        encoder_block= EncoderBlock(encoder_self_attention, feed_forward, dropout)
        encoder_blocks.append(encoder_block)

    decoder_blocks= []
    for _ in range(N):
        decoder_self_attention = MultiHeadAttention(d_model, h, dropout)
        decoder_cross_attention = MultiHeadAttention(d_model, h, dropout)
        feed_forward = FeedForward(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(decoder_self_attention, decoder_cross_attention, feed_forward, dropout)
        decoder_blocks.append(decoder_block)

    ## Creting Encoder and Decoder:
    encoder = Encoder(nn.ModuleList(encoder_blocks))
    decoder = Decoder(nn.ModuleList(decoder_blocks))

    ## Create the projection layer:
    projection = ProjectionLayer(d_model, trgt_vocab_size)

    ## Create the transformer model:
    transformer = Transformers(encoder, decoder, src_embed, trgt_embed, src_pos, trgt_pos, projection)

    ## Initialize the parameters:
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    '''
    The xavier_uniform_ function initializes the weights of the model. The weights are initialized using a uniform distribution
    '''

    return transformer

Will just use the hugging face available tokenizers for now.

In [16]:
## Loading the dataset:

dataset = load_dataset("cfilt/iitb-english-hindi")

print("Sample Data:")
print(dataset['train'][0])
print("\n")

#@ Further lookups:
print("Dataset Splits:")
print(f"Train Size: {len(dataset['train'])}")
print(f"Validation Size: {len(dataset['validation'])}")
print(f"Test Size: {len(dataset['test'])}")

README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

Sample Data:
{'translation': {'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}


Dataset Splits:
Train Size: 1659083
Validation Size: 520
Test Size: 2507


In [25]:
## Tokenization

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

def tokenize_function(examples):

    inputs = [example["en"] for example in examples["translation"]]
    targets = [example["hi"] for example in examples["translation"]]


    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="pt")


    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = dataset.map(tokenize_function, batched=True)


print("\nTokenized Sample:")
print(tokenized_dataset["train"][0])

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


KeyboardInterrupt: 

In [20]:
# Initialize Transformer model parameters
src_vocab_size = tokenizer.vocab_size
trgt_vocab_size = tokenizer.vocab_size
src_seq_len = 128
trgt_seq_len = 128

transformer = build_transformer(
    src_vocab_size=src_vocab_size,
    trgt_vocab_size=trgt_vocab_size,
    src_seq_len=src_seq_len,
    trgt_seq_len=trgt_seq_len,
    d_model=512,
    N=6,
    h=8,
    dropout=0.1,
    d_ff=2048
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformer.to(device)

Transformers(
  (encoder): Encoder(
    (layers): ModuleList(
      (0-5): 6 x EncoderBlock(
        (self_attn): MultiHeadAttention(
          (w_q): Linear(in_features=512, out_features=512, bias=True)
          (w_k): Linear(in_features=512, out_features=512, bias=True)
          (w_v): Linear(in_features=512, out_features=512, bias=True)
          (w_o): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): FeedForward(
          (linear_1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear_2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (residual_connection): ModuleList(
          (0-1): 2 x ResidualConnection(
            (dropout): Dropout(p=0.1, inplace=False)
            (norm): LayerNormalization()
          )
        )
      )
    )
    (norm): LayerNormalization()
  )
  (decoder): Decoder(
  

In [18]:
# Prepare DataLoader with padding and batching
collator = DataCollatorForSeq2Seq(tokenizer, model=None, padding=True, max_length=128)

train_dataloader = DataLoader(
    tokenized_dataset["train"],
    batch_size=16,
    shuffle=True,
    collate_fn=collator
)

val_dataloader = DataLoader(
    tokenized_dataset["validation"],
    batch_size=16,
    shuffle=False,
    collate_fn=collator
)

In [21]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = torch.optim.Adam(transformer.parameters(), lr=1e-4)

In [22]:
# Training loop
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        src = batch["input_ids"].to(device)
        trg = batch["labels"].to(device)

        src_mask = (src != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
        trg_mask = (trg != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)

        trg_input = trg[:, :-1]
        trg_output = trg[:, 1:]

        optimizer.zero_grad()
        enc_output = model.encode(src, src_mask)
        dec_output = model.decode(trg_input, enc_output, src_mask, trg_mask)
        output = model.project(dec_output)

        # Shift output for loss calculation
        loss = criterion(output.view(-1, output.size(-1)), trg_output.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [23]:
# Validation loop
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            src = batch["input_ids"].to(device)
            trg = batch["labels"].to(device)

            src_mask = (src != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
            trg_mask = (trg != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)

            trg_input = trg[:, :-1]
            trg_output = trg[:, 1:]

            enc_output = model.encode(src, src_mask)
            dec_output = model.decode(trg_input, enc_output, src_mask, trg_mask)
            output = model.project(dec_output)

            loss = criterion(output.view(-1, output.size(-1)), trg_output.view(-1))
            total_loss += loss.item()

    return total_loss / len(dataloader)


In [24]:
# Training and Evaluation
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train_epoch(transformer, train_dataloader, optimizer, criterion, device)
    val_loss = evaluate(transformer, val_dataloader, criterion, device)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`translation` in this case) have excessive nesting (inputs type `list` where type `int` is expected).